#### Load Libraries and Demo Codes

In [1]:
if True:
    from julia.api import Julia
    jl = Julia(compiled_modules=False)

import julia; julia.install(quiet=True)
from julia import Main

import sympy as sp
import numpy as np
from scipy.linalg import qr, svd
from sklearn.decomposition import PCA
from scipy.linalg import subspace_angles

import time

import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

import param
import holoviews as hv
import panel as pn

pn.extension('katex', 'plotly')
hv.extension('plotly', 'bokeh', logo=False)

%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [2]:
%%julia
using Pkg; Pkg.activate("../GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, RowEchelon, Printf, Latexify, LaTeXStrings, Random, SymPy

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


In [3]:
def qr_vs_svd_stacked_annotated(min_exp=-5, max_deg=90.0, step_exp=0.05):
    input_angles = []
    q1_u1_angles = []
    sigma_ratios = []
    r22_vals     = []

    log_degrees = np.arange(min_exp, np.log10(max_deg) + step_exp, step_exp)
    for log_deg in log_degrees:
        theta_deg = max(10 ** log_deg, 1e-15)
        theta_rad = np.radians(theta_deg)

        a1 = np.array([1.0, 0.0])
        a2 = np.array([np.cos(theta_rad), np.sin(theta_rad)])
        A  = np.column_stack((a1, a2))

        Q, R = qr(A)
        U_svd, S, _ = svd(A)

        r22_vals.append(abs(R[1, 1]))

        # angle between q₁ and u₁
        cosθ        = np.clip(np.dot(Q[:, 0], U_svd[:, 0]), -1.0, 1.0)
        angle_q1_u1 = np.degrees(np.arccos(cosθ))

        sigma_ratio = S[1] / S[0]

        input_angles.append(theta_deg)
        q1_u1_angles.append(angle_q1_u1)
        sigma_ratios.append(sigma_ratio)

    # ====================================================== Graphs
    o = hv.opts.Curve(height=120, width=500, show_grid=True,
                      #min_border=0, min_border_top=0, min_border_bottom=0,
                      logx=True, xlim=(1e-5, max_deg), yticks=4,
                      tools=['hover'])

    angle_curve = hv.Curve((input_angles, q1_u1_angles),
                           'θ(a₁, a₂) [deg]', 'θ(q₁, u₁) [deg]').opts(color='crimson', xaxis=None)

    r22_curve = hv.Curve((input_angles, r22_vals),
                         'θ(a₁, a₂) [deg]', 'R₂₂').opts( color='blue', ylabel='R₂₂', xaxis=None)

    sigma_curve = hv.Curve((input_angles, sigma_ratios),# label='computed curve')\
                           'θ(a₁, a₂) [deg]', 'σ₂ / σ₁',)\
                           .opts(color='navy')

    # Compute theoretical curve for comparison
    theta_rads         = np.radians(input_angles)
    theoretical_ratios = np.tan(theta_rads / 2)

    theoretical_curve = hv.Curve((input_angles, theoretical_ratios),
                                 'θ(a₁, a₂) [deg]', 'σ₂ / σ₁',# label='theoretical tan(θ/2)'
                                ).opts(color='green')

    # --- Annotations ---
    elbow_angle = 1.0  # where things start to diverge
    vline       = hv.VLine(elbow_angle).opts(color='gray', line_dash='dashed', line_width=2)

    label = hv.Text(elbow_angle * 1.05, 30.0, "↖ elbow region").opts(
        text_align='left', text_font_size='10pt', text_color='gray'
    )

    return pn.Column(
        (angle_curve * vline * label).opts(o).opts(height=100),
        (r22_curve * vline).opts(o).opts(height=100),
        (sigma_curve * vline ).opts(o) #.opts(legend_position='bottom')
    )

In [4]:
def generate_rotated_data(angle_deg=0, n_samples=100, noise=0.05):
    angle_rad = np.radians(angle_deg)
    rotation = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad),  np.cos(angle_rad)]
    ])
    base = np.random.randn(n_samples, 2) @ np.diag([1.0, 0.2])
    rotated = base @ rotation.T
    noisy = rotated + noise * np.random.randn(*rotated.shape)
    return noisy

# Utility: signed angle between two vectors
def signed_angle(u, v):
    angle = np.arccos(np.clip(np.dot(u, v), -1.0, 1.0))
    cross = u[0]*v[1] - u[1]*v[0]
    return np.degrees(angle) * np.sign(cross)

class PCADriftViewer(pn.viewable.Viewer):
    def __init__(self, **params):
        super().__init__(**params)

        self.angle_slider = pn.widgets.FloatSlider(name="Rotation (degrees)", width=200,
                                                   start=-90, end=90, step=1, value=0)
        self.metric_selector = pn.widgets.Select(name="Metric", options=["Angle", "Spectral", "Chordal", "Geodesic"],
                                                 width=120)

        self.plot_pane = pn.pane.HoloViews(height=350, width=350)
        self.info_pane = pn.pane.Markdown(width=280)
        self.metric_curve_pane = pn.pane.HoloViews(height=250, width=280)

        self.reset_button = pn.widgets.Button(name="Reset", width=100)
        self.reset_button.on_click(self._on_reset)

        self._init_data()
        self.angle_slider.param.watch(self._update, 'value')
        self.metric_selector.param.watch(self._update, 'value')

        self._update()

    def _init_data(self):
        self.start_time = time.time()
        self.elapsed_times = []
        self.angle_history = []
        self.spectral_history = []
        self.chordal_history = []
        self.geodesic_history = []

        self.initial_data = generate_rotated_data(0)
        self.U0 = PCA(n_components=1).fit(self.initial_data).components_.T

    def _on_reset(self, *events):
        self._init_data()
        self._update()

    def _update(self, *events):
        theta = self.angle_slider.value
        data = generate_rotated_data(theta)
        U_t = PCA(n_components=1).fit(data).components_.T

        # Time tracking
        now = time.time()
        elapsed = now - self.start_time
        self.elapsed_times.append(elapsed)

        # Distance metrics
        angle_deg = signed_angle(self.U0[:, 0], U_t[:, 0])
        angles_rad = subspace_angles(self.U0, U_t)

        spectral = np.degrees(angles_rad[0])
        chordal  = np.linalg.norm(np.sin(angles_rad))
        geodesic = np.linalg.norm(np.degrees(angles_rad))

        self.angle_history.append(angle_deg)
        self.spectral_history.append(spectral)
        self.chordal_history.append(chordal)
        self.geodesic_history.append(geodesic)

        # Visual arc
        arc_theta = np.linspace(0, np.radians(angle_deg), 50)
        arc_x = np.cos(arc_theta)
        arc_y = np.sin(arc_theta)
        arc = hv.Curve((arc_x, arc_y)).opts(color='red', line_width=2)

        arrow_U0 = hv.Segments([(0, 0, *self.U0[:, 0])]).opts(line_color='gray', line_width=3)
        arrow_Ut = hv.Segments([(0, 0, *U_t[:, 0])]).opts(line_color='red', line_width=3)
        scatter  = hv.Points(data).opts(size=4, color='blue', alpha=0.5)

        plot = (scatter * arrow_U0 * arrow_Ut * arc).opts(
            aspect='equal', xlim=(-3, 3), ylim=(-3, 3), title="PCA Drift"
        )
        self.plot_pane.object = plot

        self.info_pane.object = f"""
<span style="font-size:14px">
<b>Principal Angle</b>: {angle_deg:+.2f}°<br>
<b>Rotation Input</b>: {theta:+.0f}°<br><br>
- Red: current PCA direction<br>
- Red arc: signed principal angle
</span>
"""

        if len(self.elapsed_times) >= 2:
            selected = self.metric_selector.value
            if selected == "Angle":
                yvals = self.angle_history
                ylabel = "Angle [deg]"
            elif selected == "Spectral":
                yvals = self.spectral_history
                ylabel = "Spectral Distance [deg]"
            elif selected == "Chordal":
                yvals = self.chordal_history
                ylabel = "Chordal Distance"
            else:
                yvals = self.geodesic_history
                ylabel = "Geodesic Distance [deg]"

            curve = hv.Curve((self.elapsed_times, yvals), 'Time [s]', ylabel).opts(
                color='crimson', height=250, tools=['hover'], show_grid=True,
                title=f"{selected} vs Time"
            )
            self.metric_curve_pane.object = curve

    def __panel__(self):
        return pn.Column(
            "## PCA Drift Demo (Metric Tracking)",
            pn.Row(self.reset_button, self.angle_slider, pn.Spacer(width=50), self.metric_selector),
            pn.Row(
                self.plot_pane,
                pn.Spacer(width=20),
                pn.Column(
                    pn.Spacer(height=10),
                    self.info_pane,
                    self.metric_curve_pane
                )
            )
        )


# 
<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Computing Principal Angles</strong><br>
<strong style="height:100px;color:darkred;font-size:25px;">Algorithms and Selected Examples</strong></div>

# 1. Review: Principal Angles Between Subspaces

**Previous notebooks in this series:**
* [**KplanesPrincipalAngles.ipynb**](KplanesPrincipalAngles.ipynb)
* [**KplaneDistances.ipynb**](KplaneDistances.ipynb)

**Principal Angles Between Subspaces**

* Let $\mathscr{C}(U), \mathscr{C}(V) \subset \mathbb{R}^n$ be subspaces of dimension $k$ and $\ell$, respectively.<br>
The **principal angles** $\theta_1 \leq \cdots \leq \theta_r$, where $r = \min(k, \ell)$,<br>
measure the smallest angles between directions in the two subspaces.

* If the columns of both $U$ and $V$ are mutually orthogonal and unit length,<br>
i.e., if they are **orthonormal basis matrices**,<br>
then the principal angles are determined by the **singular values of** $\mathbf{U^T V}$<br><br>
If $\{ \sigma_i, i=1,\dots,r \}\;$ are the singular values of $U^T V$, the principal angles are given by<br>
$\qquad \theta_i = \cos^{-1}(\sigma_i)$.

This formulation is the most practical and numerically stable for computation.

---
**Focus of This Notebook**

This notebook examines the **algorithmic and numerical aspects** of principal angle computation. We go beyond the core definition to explore:

- The importance of **orthonormalizing basis matrices**
- Comparing **QR and SVD orthonormalization** strategies
- Behavior under **near-degenerate or ill-conditioned subspaces**
- The effect of numerical precision on **subspace distance metrics**

Small-scale demos, reproducible examples, and visual diagnostics are included to highlight robustness (or failure) in practice.

# 2. Algorithmic Choices in Computing Principal Angles

To compute principal angles between two subspaces with bases $A, B \in \mathbb{R}^{n \times k}$,<br>
we must **orthonormalize** the columns.

A poor orthonormalization step can distort angles — especially the small ones<br>
as we have already seen in the previous notebook.

## 2.1 Orthonormalization

### 2.1.1 Option 1: QR Decomposition

Fast and effective for well-conditioned matrices: compute the QR decomposition of the matrices $A$ and $B$.<br>
This is fast and numerically stable.

* $\qquad$ Decompose $\;A = Q R\;$ and set $U = Q$.
* $\qquad$ Decompose $\;B = Q R\;$ and set $V = Q$.

<div style="display: flex; margin-top: 1em; margin-bottom: 1em;">
<!-- ----------------------------------------- Python box -->
<div style="width: 22em; height: 4.8em; padding-right: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Python</div>
    <div>U, _ = np.linalg.qr(A)</div>
    <div>V, _ = np.linalg.qr(B)</div>
  </div>
</div>
<!-- ---------------------------------------- Julia box -->
<div style="width: 22em; height: 4.8em; margin-left: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Julia</div>
    <div>U = qr(A).Q</div>
    <div>V = qr(B).Q</div>
  </div>
</div>
</div>

### 2.1.2 Option 2: SVD-Based Orthonormalization

In this approach, we compute the **singular value decomposition** of the input matrices $A$ and $B$, and extract the left singular vectors to obtain orthonormal bases for their column spaces.

This method is **more robust** than QR when $A$ or $B$ are nearly rank-deficient or ill-conditioned. It is especially useful when numerical precision matters in detecting small principal angles.

* Decompose $\;A = U \Sigma V^\top\;$ and set $U_{\text{orth}} = U$.
* Decompose $\;B = U \Sigma V^\top\;$ and set $V_{\text{orth}} = U$.

<div style="display: flex; margin-top: 1em; margin-bottom: 1em;">

<!-- Python box -->
<div style="width: 10cm; height: 4.8em; padding-right: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Python</div>
    <div>U, _, _ = np.linalg.svd(A, full_matrices=False)</div>
    <div>V, _, _ = np.linalg.svd(B, full_matrices=False)</div>
  </div>
</div>

<!-- Julia box -->
<div style="width: 8cm; height: 4.8em; margin-left: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Julia</div>
    <div>U, _, _ = svd(A, full=false)</div>
    <div>V, _, _ = svd(B, full=false)</div>
  </div>
</div>
</div>

### 2.1.3 Numerical Rank and QR Pitfalls

Principal angle computations begin with $k$ vectors assumed to span a $k$-dimensional subspace.

If the vectors are nearly dependent — or the rank is unknown — the matrix may be numerically rank-deficient.

* **QR** always returns $k$ orthonormal vectors, assuming full rank.<br>
The first may align with the true subspace, but later vectors are orthogonalized without regard to geometry.

* **By contrast, SVD detects the true rank and returns geometrically meaningful directions.**<br>
It identifies stable components of the column space, even when directions are nearly aligned or ill-conditioned.

We will look an example to clarify this behavior.

#### **Example: Redundant Vectors in 3D**

Let $V \in \mathbb{R}^{3×3}$ contain three vectors:
- Vectors $v_1$ and $v_3$ **lie in the xy-plane**
- Vector $v_2$ **lies nearly in the y plane**

$\qquad V= \left(\begin{array}{rrr}
1.0 & -0.01 & 0.0 \\
0.0 & 0.01 & 1.0 \\
0.0 & 0.03 & 0.0 \\
\end{array}\right)$

In [5]:
%%julia
V = [1 -0.01  0;
     0  0.01  1;
     0  0.03  0]

QR = qr(V)
Q_v = Matrix(QR.Q)

display(py_show(L"\text{QR yields}\;\;Q_v =", Q_v, L", \qquad \text{diag}(R) =", diag(QR.R), number_formatter=x->round_value(x, 2)))

SVD = svd(V)
V_svd = SVD.U
println()
display(py_show("By contrast, the SVD achieves the desired geometrical description: the xy plane and a z axis vector with small sigma value"))
py_show( L"\qquad V_{svd} =", V_svd, L",\qquad \sigma=", SVD.S, number_formatter=x->round_value(x,2))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

____
QR normally processes the vectors one at a time: the first two vectors lie in the span$\{v_1, v_2\}$<br>
$\qquad$ loosing any notion of an xy plane $\mathcal{U}$:<br>
$\qquad$ $q_1$ defines the x axis, but both $q_2$ and $q_3& have significant z components!

____
We can actually see the problem by investigating the diagonal entries of $R$:<br>
Each diagonal entry $R_{ii}$ in QR reflects the norm of the component added at step $i$.
- $R_{1 1} = \Vert v_1\Vert$
- $R_{2 2} = \Vert v_2 - \text{proj}_{q_1} v_2\Vert$
- $\dots$.

**Small $R_{i i}$ values indicate a direction nearly contained in the span of previous vectors.**<br>
Such components contribute little new geometric information: they do however result in significant distortion.

Removing the column with the smallest $R_{i i}$ **does not guarantee a valid geometric basis.**
The remaining vectors may still be misaligned with the underlying subspace.

Instead, choose to reorder the input columns so that the $R_{i i}$ decrease in magnitude with increasing $i$.<br>
This ensures that the most significant directions appear first, and weaker, unstable directions can be safely truncated.

In [6]:
%%julia
QR_ordered = qr(V, ColumnNorm())
Q_ordered_v = Matrix(QR_ordered.Q)
display(py_show( L"\text{QR with column reordering}\;\; Q_v = ", Q_ordered_v, L", \quad \text{Diagonal terms  diag}(R) = ", diag(QR_ordered.R), number_formatter=x->round_value(x,2)))

<IPython.core.display.Latex object>

**The first two vectors now recover the xy plane.** The third vector has a small $R_{3 3}$ value,<br>
$\qquad$ showing that it can be discarded.

## 2.2 Principal Angle Computation

Now that we have orthonormal bases, i.e., the cols of $U \in \mathbb{R}^{M\times p},  \in \mathbb{R}^{M\times p}$,<br>
the principal angles are computed by a single SVD of $U^T V$.

$\qquad
\cos \theta_i = \sigma_i(U^T V)
$

These angles immediately determine all standard subspace distances (e.g., spectral, chordal, geodesic),<br>
as detailed in the [**previous notebook**](KplaneDistances.ipynb).


This is efficient, numerically stable, and widely used.

<div style="display: flex; margin-top: 1em; margin-bottom: 1em;">

<!-- Python box -->
<div style="width: 11.5cm; height: 10.8em; padding-right: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.4;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Python</div>
    <div>def principal_angles(U::AbstractMatrix, V::AbstractMatrix):</div>
    <div>&nbsp;&nbsp;&nbsp;&nbsp;Σ = np.linalg.svd(U.T @ V, compute_uv=False)</div>
    <div>&nbsp;&nbsp;&nbsp;&nbsp;θ = np.arccos(np.clip(Σ, -1, 1))</div>
    <div>&nbsp;&nbsp;&nbsp;&nbsp;return np.degrees(θ)</div>
  </div>
</div>

<!-- Julia box -->
<div style="width: 12.5cm; height: 10.8em; margin-left: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.4;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Julia</div>
    <div>function principal_angles(U::AbstractMatrix, V::AbstractMatrix)</div>
    <div>&nbsp;&nbsp;&nbsp;&nbsp;S = svd(U' * V).S</div>
    <div>&nbsp;&nbsp;&nbsp;&nbsp;clamped = clamp.(S, -1.0, 1.0)</div>
    <div>&nbsp;&nbsp;&nbsp;&nbsp;return acosd.(clamped)</div>
    <div>end</div>
  </div>
</div>

</div>

#### Example continued: Effect on Computing Principal Angles

<div style="float:left;padding-top:1cm;padding-left:0.5cm;">

Let's continue the example by computing<br>
the **principal angles of this example**<br>
**with respect to a plane tilted by 20 degrees**
* using the SVD
* using QR
* using QR and removing the second vector based on its $R_{2 2} value
* using QR with column reordering
</div>
<div style="float:left;padding-left:2cm;">
    <img src="Figs/PrincipalAngles_2planes.png" width=300></div>
</div>

In [7]:
%%julia
function principal_angles_using_svd(U::AbstractMatrix, V::AbstractMatrix)
    S = svd(U' * V).S
    clamped = clamp.(S, -1.0, 1.0)
    return acosd.(clamped)
end

# -- Distance metrics from principal angles
function distance_metrics(θ)
    sinθ = sin.(θ)
    return Dict(
        "spectral"  => maximum(sinθ),
        "chordal"   => norm(sinθ),
        "frobenius" => sqrt(2) * norm(sinθ),
        "geodesic"  => norm(θ)
    )
end

function angle_to_vector(θ_deg, base, direction)
    θ = deg2rad(θ_deg)
    return cos(θ) * base + sin(θ) * direction
end
# ####################################################################################################
function example_principal_angles()
    # ####################################################################################################
    # U:  a plane tilted 20 degrees with respect to the xy plane
    # two vectors in the xy plane
    α = 20
    Ry = [cosd(α) 0   sind(α);
          0       1   0;
         -sind(α) 0   cosd(α)]
    Q_tilted = Matrix(qr([Ry * [1,0,0] Ry * [1,2,0]]).Q)

    # ####################################################################################################
    # Compute the basis for each of the methods
    # Method 1: SVD of V
    SVD = svd(V)
    Q1_svd = SVD.U[:, 1:2]

    # Method 2: QR (unpivoted)
    QR    = qr(V)
    Q1_qr = Matrix(QR.Q)
    diagR = diag(QR.R)

    # Method 3: QR with second vector removed
    Q1_qr_drop = Q1_qr[:, [1,3]]

    # Method 4: Pivoted QR
    QR_pivoted    = qr(V, ColumnNorm())
    Q1_qr_pivoted = Matrix(QR_pivoted.Q[:, 1:2])

    # ####################################################################################################
    # Compute the principal angles for each of the methods
    # Compute all principal angles
    angles = Dict(
        "SVD"      => principal_angles_using_svd(Q1_svd,        Q_tilted),
        "QR"       => principal_angles_using_svd(Q1_qr,         Q_tilted),
        "QR drop"  => principal_angles_using_svd(Q1_qr_drop,    Q_tilted),
        "QR pivot" => principal_angles_using_svd(Q1_qr_pivoted, Q_tilted),
    )
    # Display formatted table
    method_order = ["SVD", "QR", "QR drop", "QR pivot"]

    # Display formatted table with fixed order
    println("Method       | Angle 1 (°) | Angle 2 (°)")
    println("-------------|-------------|------------")
    for method in method_order
        θs = angles[method]
        @printf("%-12s | %11.2f | %10.2f\n", method, θs[1], θs[2])
    end
    println()
end
example_principal_angles()

Method       | Angle 1 (°) | Angle 2 (°)
-------------|-------------|------------
SVD          |        0.00 |      19.98
QR           |        0.00 |       0.00
QR drop      |        0.00 |      26.94
QR pivot     |        0.00 |      20.00


#### Recommendation

Both **SVD** and **pivoted QR** reveal the intrinsic dimensionality of a subspace by ordering directions by importance.<br>
However, **neither method removes vectors automatically**<br>
$\qquad$ a threshold on singular values ($\sigma_i$) or diagonal QR entries ($R_{ii}$) is required to discard unstable components.

**SVD** and **pivoted QR**, when combined with appropriate truncation, reliably recover the geometry of nearly dependent subspaces.


____
**QR** is fast and reliable when the input matrix has full rank and well-separated directions.<br>
$\qquad$ It returns an orthonormal basis where the first vector matches the first input column.<br>
$\qquad$ **Use QR for speed and simplicity when the geometry is well-conditioned.**

**SVD** gives a geometrically optimal basis, selecting directions<br>
$\qquad$ that span the column space as faithfully as possible.<br>
$\qquad$ It is especially useful when vectors are nearly dependent or rank-deficient.<br>
$\qquad$ **Use SVD when precision in subspace alignment matters, especially in degenerate or near-degenerate cases.**

## 2.4 Theoretical Behavior of a 2D Example

$\qquad$ Let $a_1, a_2 \in \mathbb{R}^n$, unit vectors with angle $\theta$ and set $A = (a_1 \;\; a_2) = \begin{pmatrix}1 & \cos\theta \\ 0 & \sin\theta \end{pmatrix}$.

$\qquad$ **QRdecomposition**<br>
$\qquad$ $\qquad A = \left(\begin{array}{rr}1 & 0 \\ 0 & 1 \end{array}\right)
\begin{pmatrix}1 & \cos\theta \\ 0 & \sin\theta \end{pmatrix},\qquad$ thus $R_{2 2} = \sin\theta$ 

$\qquad$ **Principal Values**<br>
$\qquad$ $\qquad A^T A = \begin{pmatrix} 1 & \cos\theta \\ \cos\theta & 1 \end{pmatrix},\quad$
has eigenvalues $\lambda = 1 \pm \cos\theta$  

$\qquad$ $\qquad$ Thus $\frac{\sigma_2}{\sigma_1} = \sqrt{\frac{1 - \cos\theta}{1 + \cos\theta}} = \tan(\theta/2).\qquad$  (Note this is the inverse of the condition number)

In [8]:
hv.extension('bokeh', logo=False)
pn.Column(
    "# QR vs SVD Diagnostics as Vector Angle Increase",
    qr_vs_svd_stacked_annotated()
).servable()

Column
    [0] Markdown(str)
    [1] Column
        [0] HoloViews(Overlay, height=100, sizing_mode='fixed', width=500)
        [1] HoloViews(Overlay, height=100, sizing_mode='fixed', width=500)
        [2] HoloViews(Overlay, height=120, sizing_mode='fixed', width=500)

**Conclusion**

As the angle $\theta$ between $a_1$ and $a_2$ increases, the subspace becomes more clearly 2-dimensional.  

- **QR** always returns $q_1 = a_1 / \|a_1\|$, preserving the input order.
- **SVD** selects $u_1$, the direction that best spans the full column space of $A$.

At small $\theta$, $a_1$ and $a_2$ are nearly aligned, so $q_1 \approx u_1$, and $\sigma_2 \ll \sigma_1$, $R_{22} \approx 0$.

As $\theta$ grows, the second direction becomes more significant:
- $u_1$ rotates to balance both vectors, reflecting the growing importance of the second direction
- $\angle(q_1, u_1)$ increases,
- $\sigma_2 / \sigma_1$ and $R_{22}$ both increase.

**Takeaway:**<br>
$\qquad$ R and SVD agree in the well-separated regime — but diverge geometrically when vectors are nearly dependent.

$\qquad$ For subspaces defined by nearly aligned vectors, **SVD more accurately captures the dominant direction**,<br>$\qquad$  while QR reflects the first input vector.<br>  
$\qquad$ As the separation grows, both methods yield similar bases.

# 3. Application

When applying PCA to streaming or time-evolving data, it is often useful to track how the dominant subspace changes over time.

Given a sequence of datasets $X_t \in \mathbb{R}^{m \times n}$, we compute PCA at each timestep to extract a $k$-dimensional subspace $U_t$.

To monitor drift, we compute the **principal angles** between $U_t$ and $U_{t+1}$. These angles — or distances derived from them — provide a quantitative measure of how much the underlying structure is changing.

For example:
- A small spectral distance $\theta_1$ indicates the top direction is stable.
- A large chordal or geodesic distance may indicate broader change.

This forms the basis for **concept drift detection**, **subspace tracking**, and **change point analysis**<br> in applications like signal processing, finance, and machine learning.

Below we construct a toy example showing how the distance between PCA subspaces changes as the data distribution rotates slowly.

In [9]:
hv.extension('bokeh', logo=False)
demo = PCADriftViewer()
demo.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] Button(name='Reset', width=100)
        [1] FloatSlider(end=90, name='Rotation (degrees)', start=-90, step=1, width=200)
        [2] Spacer(width=50)
        [3] Select(name='Metric', options=['Angle', 'Spectral', ...], value='Angle', width=120)
    [2] Row
        [0] HoloViews(Overlay, height=350, sizing_mode='fixed', width=350)
        [1] Spacer(width=20)
        [2] Column
            [0] Spacer(height=10)
            [1] Markdown(str, width=280)
            [2] HoloViews(None, height=250, sizing_mode='fixed', width=280)

**Example Applications using Subspace Drift**

Monitoring changes in principal angles between PCA subspaces provides a quantitative measure of structural shifts in evolving data. This approach is instrumental in various domains:

- **Streaming Data & Online Learning**: Detecting concept drift in real-time systems, such as recommendation engines and spam filters, where user behavior evolves over time.
- **Video Surveillance & Background Modeling**: Identifying scene changes or motion by tracking subspace variations in video frames.
- **Neuroscience & Biomedical Signals**: Monitoring shifts in EEG or fMRI data to detect phase transitions or anomalies in brain activity.
- **Financial Time Series Analysis**: Observing changes in market dynamics by analyzing shifts in principal components over different time periods.

# 4. Take Away

Principal angles offer a stable, geometric way to compare subspaces — but their accuracy depends on how those subspaces are constructed.

* **QR** assumes full rank and may introduce spurious directions when vectors are nearly dependent.
* **SVD** reveals the true numerical rank and isolates meaningful structure.

In ill-conditioned settings or when rank is uncertain, **SVD or pivoted QR with truncation** <br>
provides a more faithful representation of the subspace geometry.